In [3]:
module Compositions

compositions = (;
    co2impure="REFPROP::CarbonDioxide[0.969696970]&Hydrogen[0.010101010]&Nitrogen[0.010101010]&Argon[0.009090909]&CarbonMonoxide[0.001010101]",
    co2pure="CarbonDioxide[1.0]"
)

export compositions

end

Main.Compositions

In [49]:
using .Compositions
using CoolProp
using Unitful
using ModelingToolkit
using DifferentialEquations

@variables t

1-element Vector{Num}:
 t

In [ ]:
println(compositions.co2impure)

In [ ]:
function eos(composition::String, p, T) 
    p - ρ * T * 287.11
end

In [ ]:
eos(compositions.co2pure, 10u"bar", 293.15u"K")

In [52]:


"""

State Equation of Dry Air
"""
stateEquation(p, ρ, T) = p - ρ * T * 287.11
@register_symbolic stateEquation(p, ρ, T) #状态方程


"""

Port(inlet or outlet) in CompressedAirSystem.

# States:
- `p(t)`: [`Pa`] The pressure at this port
- `qm(t)`: [`kg/s`] The mass flow rate passing through this port
- `T(t)`: [`K`] The mass flow rate passing through this port
- `ρ(t)`: [`kg/m³`] The density passing at this port
- `μ(t)`: [`Pa⋅s`] The absolute viscosity at this port
- `qv(t)`: [`m³/s`] The volume  flow rate passing through this port

"""
@connector function FlowPort(; name)
    sts = @variables begin
        p(t) = 1.013e5
        T(t) = 300
        (qm(t)=0, [connect = Flow])
        (ρ(t)=1.2, [connect = Stream])
        (μ(t)=1.819e-5, [connect = Stream])
        (qv(t)=0, [connect = Stream])
    end
    eqs = [
        0 ~ stateEquation(p, ρ, T)
        μ ~ 1.819e-5
        qv ~ qm / ρ
    ]
    ODESystem(eqs, t, sts, []; name=name)
end

FlowPort

In [49]:
@connector function FluidFlowPort(composition::String ; name)
    sts = @variables begin
        p(t) = 1.013e5
        T(t) = 300
        (qm(t)=0, [connect = Flow])
        (ρ(t)= PropsSI("D", "P", p, "T", T, composition), [connect = Stream])
        (μ(t)= PropsSI("V", "P", p, "T", T, composition), [connect = Stream])
        (qv(t)=0, [connect = Stream])
    end
    eqs = [
        0 ~ stateEquation(p, ρ, T)
        μ ~ 1.819e-5
        qv ~ qm / ρ
    ]
    ODESystem(eqs, t, sts, []; name=name)
end

FluidFlowPort (generic function with 1 method)

In [53]:
@variables t
@named ffp = FluidFlowPort(compositions.co2pure)

┌ Warning: ffp contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40


Model ffp with 3 (9) equations
States (6):
  p(t) [defaults to 101300.0]
  T(t) [defaults to 300]
  qm(t) [defaults to 0]
  ρ(t) [defaults to PropsSI(D, P, p(t), T, T(t), CarbonDioxide[1.0])]
⋮
Parameters (0):

In [ ]:
@named model = compose(ODESystem(eqs, t, name=:funs), components)

sys = structural_simplify(model)

# next section

In [72]:

@register_symbolic CoolProp.:PropsSI(out::AbstractString, name1::AbstractString, value1::Real, name2::AbstractString, value2::Real, fluid::AbstractString)


const gn = Unitful.gn.val

function _NodeEnergy(node, D, ρ)
    return node.p / (ρ * gn) + node.z + 8 * abs2(node.q) / (π^2 * D^4 * gn)
end

"""
To get the loss of resistance along the pipe(between two ports).

In volume flow form:

```math
h_f = f\\frac{L}{D} \\frac{8q^{2}}{\\pi^2D^4g}
```

"""
function _h_f(node, f, L, D)
    return f * L * 8 / (D^5 * π^2 * gn) * node.q * abs(node.q)
end


"""
To get the local resistance loss the components.

In volume flow form:

```math
h_m = K \\frac{8q^{2}}{\\pi^2D^4g}
```

"""
function _h_m(node, K, D)
    return K * 8 / (D^4 * π^2 * gn) * node.q * abs(node.q)
end

function Sink_P(; name, p=101325)
    @named port = PipeNode2(z=0.0)
    eqs = [
        port.p ~ p
    ]
    compose(ODESystem(eqs, t, [], [], name=name), port)
end

function FP(; name, P=1.013e5, Tm=300, L=10.0, D=1.0, f=0.01, K_inside=0.0)
    @named in = PipeNode2()
    @named out = PipeNode2()
    ps = @parameters L=L D=D f=f K_inside=K_inside u=20
    @variables begin
        Aₛ
        p = P
        tm(t) = Tm
        ρ
    end
    eqs = [
        Aₛ ~ π * D * L,
        ρ ~ PropsSI("D", "P", p, "T", Tm, compositions.co2pure),
        p ~ P,
        _NodeEnergy(in, D, ρ) ~ _NodeEnergy(out, D, ρ) + _h_f(in, f, L, D) + _h_m(in, K_inside, D) + u * Aₛ,
        0 ~ in.q + out.q
        ]
    
    ODESystem(eqs, t, [], ps; name=name)
    compose(ODESystem(eqs, t, [], ps, name=name), in, out)
end

FP (generic function with 1 method)

In [73]:
@named test = FP()

Model test with 5 (7) equations
States (4):
  in₊p(t) [defaults to 1.0]
  in₊q(t) [defaults to 1.0]
  out₊p(t) [defaults to 1.0]
  out₊q(t) [defaults to 1.0]
Parameters (7):
  L [defaults to 10.0]
  D [defaults to 1.0]
  f [defaults to 0.01]
  K_inside [defaults to 0.0]
⋮

In [58]:
@named low = Sink_P(p=100000)

Model low with 1 (2) equations
States (2):
  port₊p(t) [defaults to 1.0]
  port₊q(t) [defaults to 1.0]
Parameters (1):
  port₊z [defaults to 0.0]

In [67]:
eqs = [connect(test.out, low.port)]

1-element Vector{Equation}:
 connect(test.out, low.port)

In [74]:
@named model = compose(ODESystem(eqs, t, name=:funs), [test, low])


Model model with 6 (9) equations
States (6):
  test₊in₊p(t) [defaults to 1.0]
  test₊in₊q(t) [defaults to 1.0]
  test₊out₊p(t) [defaults to 1.0]
  test₊out₊q(t) [defaults to 1.0]
⋮
Parameters (8):
  test₊L [defaults to 10.0]
  test₊D [defaults to 1.0]
  test₊f [defaults to 0.01]
  test₊K_inside [defaults to 0.0]
⋮

In [75]:
sys = structural_simplify(model)
prob = ODEProblem(sys, [], (0.0, 0.0))
sol = ModelingToolkit.:solve(prob)

retcode: Success
Interpolation: specialized 4rd order "free" stiffness-aware interpolation
t: 1-element Vector{Float64}:
 0.0
u: 1-element Vector{Vector{Float64}}:
 [111067.5874933406]

In [76]:
sol[test.out.p]

1-element Vector{Float64}:
 100000.0

In [11]:
@connector function PipeNode2(; name, z=0)
    sts = @variables begin
        p(t) = 1.0
        (q(t)=1.0, [connect = Flow])
    end
    ps = @parameters z = z
    ODESystem(Equation[], t, sts, ps; name=name)
end

PipeNode2 (generic function with 1 method)